# **Prophet_no_feature**

In [ ]:
!pip uninstall -y prophet pystan
!pip install prophet==1.1.5
!pip install cmdstanpy==1.2.2

Found existing installation: prophet 1.1.5
Uninstalling prophet-1.1.5:
  Successfully uninstalled prophet-1.1.5
  Using cached prophet-1.1.5-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.5 kB)
Using cached prophet-1.1.5-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.4 MB)


In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet

# ----------------------------------------------------
# 1) 일매출 정제 함수
# ----------------------------------------------------
def clean_sales(df, col='일매출'):
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(',', '')
        .str.replace(' ', '')
        .str.strip()
    )
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

# ----------------------------------------------------
# 2) Prophet 학습
# ----------------------------------------------------
def train_prophet(train_df):
    train_df = clean_sales(train_df, '일매출')

    df = train_df.rename(columns={'영업일자': 'ds', '일매출': 'y'})
    df['ds'] = pd.to_datetime(df['ds'])

    model = Prophet(
        weekly_seasonality=True,
        daily_seasonality=False,
        yearly_seasonality=False,
        stan_backend='CMDSTANPY'
    )
    model.fit(df)
    return model

# ----------------------------------------------------
# 3) Prophet 예측
# ----------------------------------------------------
def predict_prophet(model, periods):
    future = model.make_future_dataframe(periods=periods, freq='D')
    forecast = model.predict(future)

    pred = forecast[['ds', 'yhat']].tail(periods)
    pred = pred.rename(columns={'ds': '영업일자', 'yhat': '예측매출'})

    pred['예측매출'] = pd.to_numeric(pred['예측매출'], errors='coerce').fillna(0)

    # 🔥 음수 예측 방지 로직 추가
    pred['예측매출'] = pred['예측매출'].apply(lambda x: max(x, 0))

    return pred.reset_index(drop=True)

# ----------------------------------------------------
# 4) 평가 지표
# ----------------------------------------------------
def smape(a, f):
    a = np.array(a)
    f = np.array(f)
    denom = (np.abs(a) + np.abs(f)) / 2
    mask = denom != 0
    return np.mean(np.abs(a[mask] - f[mask]) / denom[mask]) * 100

def evaluate(test_df, pred_df):
    test_df = clean_sales(test_df, '일매출')
    pred_df['예측매출'] = pd.to_numeric(pred_df['예측매출'], errors='coerce').fillna(0)

    merged = test_df.merge(pred_df, on='영업일자', how='left')

    actual = merged['일매출'].astype(float)
    pred   = merged['예측매출'].astype(float)

    mae = np.mean(np.abs(actual - pred))
    rmse = np.sqrt(np.mean((actual - pred)**2))
    smape_v = smape(actual, pred)

    return mae, rmse, smape_v

# ----------------------------------------------------
# 🔥 메인 실행
# ----------------------------------------------------
train = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_train_val.csv')
test  = pd.read_csv('/content/drive/MyDrive/기계학습/팀플/Data/POS_test.csv')

train['영업일자'] = pd.to_datetime(train['영업일자'])
test['영업일자']  = pd.to_datetime(test['영업일자'])

train = clean_sales(train)
test  = clean_sales(test)

# Prophet 학습 (train 전체 사용)
model = train_prophet(train)

# Test 예측
test_pred_df = predict_prophet(model, periods=len(test))

# 🔥 날짜를 test 날짜로 강제 맞추기
test_pred_df['영업일자'] = test['영업일자'].values

# Test 평가
test_mae, test_rmse, test_smape = evaluate(test, test_pred_df)

print("\n------ Test 성능 (최종) ------")
print(f"MAE   : {test_mae:.3f}")
print(f"RMSE  : {test_rmse:.3f}")
print(f"SMAPE : {test_smape:.3f}%")

print("\n------ Test 예측 결과 ------")
print(test_pred_df)


DEBUG:cmdstanpy:input tempfile: /tmp/tmpqfgo6p8i/bm1yd2cs.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqfgo6p8i/u8e30e2f.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=93066', 'data', 'file=/tmp/tmpqfgo6p8i/bm1yd2cs.json', 'init=/tmp/tmpqfgo6p8i/u8e30e2f.json', 'output', 'file=/tmp/tmpqfgo6p8i/prophet_model6jx7zuwq/prophet_model-20251115141950.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:19:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:19:50 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



------ Test 성능 (최종) ------
MAE   : 92637.830
RMSE  : 121136.077
SMAPE : 77.454%

------ Test 예측 결과 ------
         영업일자           예측매출
0  2025-08-10       0.000000
1  2025-08-11  237594.618752
2  2025-08-12  260073.336491
3  2025-08-13  254414.054339
4  2025-08-14  247845.017839
..        ...            ...
78 2025-10-27  226805.809557
79 2025-10-28  249284.527296
80 2025-10-29  243625.245143
81 2025-10-30  237056.208644
82 2025-10-31  193068.148506

[83 rows x 2 columns]
